In [1]:
# Trains the one-shot learning model
# Adapted from https://github.com/hlamba28/One-Shot-Learning-with-Siamese-Networks to suit my 224x224x3 data
# Modifications include redesigning the model architecture

import sys
import numpy as np
import pandas as pd
import pickle
import os
import matplotlib.pyplot as plt
%matplotlib inline

import cv2
import time

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from tensorflow.keras.models import Model

from tensorflow.keras.layers import BatchNormalization, MaxPooling2D, Concatenate
from tensorflow.keras.layers import Lambda, Flatten, Dense, Dropout
from tensorflow.keras.initializers import glorot_uniform

from tensorflow.keras.layers import Layer
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K

from sklearn.utils import shuffle

import numpy.random as rng

In [2]:
from tensorflow.keras.applications.densenet import preprocess_input
data = np.load('../df6.npz', allow_pickle=True)
data = dict(data)

Xtrain = data['x_train']
train_classes = data['y_train']
Xv = data['x_test']
val_classes = data['y_test']
Xtrain = preprocess_input(Xtrain)
Xval = preprocess_input(Xv)
print(Xtrain.shape)

(23, 174, 224, 224, 3)


In [3]:
Xte = data['x_test']
test_classes = data['y_test']
Xtest = preprocess_input(Xte)

In [4]:
ux = []
uy = []
for i in range(5):
    uy.extend([i] * 20)
    
tpath = '../unseen'
for dir in os.listdir(tpath):
    path = os.path.join(tpath, dir)
    if not os.path.isdir(path): continue
    fns = os.listdir(path)
    imgs = []
    for fn in fns:
        img = cv2.imread(os.path.join(path, fn))
        if img is None: continue
        imgs.append(img)
    ux.append(np.array(imgs))

ux = np.array(ux)
uy = np.array(uy)
print(ux.shape, uy.shape)
uxp = preprocess_input(ux)

(5, 20, 224, 224, 3) (100,)


In [36]:
m = tf.keras.models.load_model('../model/dense2')

In [5]:
def initialize_weights(shape, dtype, name=None):
    """
        The paper, http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
        suggests to initialize CNN layer weights with mean as 0.0 and standard deviation of 0.01
    """
    return np.random.normal(loc = 0.0, scale = 1e-2, size = shape)

In [6]:
def initialize_bias(shape, dtype, name=None):
    """
        The paper, http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
        suggests to initialize CNN layer bias with mean as 0.5 and standard deviation of 0.01
    """
    return np.random.normal(loc = 0.5, scale = 1e-2, size = shape)

In [7]:
from tensorflow.keras.models import Model
from tensorflow.keras import layers, regularizers, optimizers
from tensorflow.keras.applications.densenet import DenseNet121 #resnet_v2 import ResNet50V2
from tensorflow.keras.constraints import max_norm
def get_siamese_model(input_shape):
    """
        Model architecture based on the one provided in: http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
    """
    
    # Define the tensors for the two input images
    left_input = Input(input_shape)
    right_input = Input(input_shape)
    
    # Convolutional Neural Network
    model = Sequential()
    model.add(Conv2D(64, (10,10), activation='relu', input_shape=input_shape,
                   kernel_initializer=initialize_weights, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (7,7), activation='relu',
                     kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (4,4), activation='relu', kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(256, (4,4), activation='relu', kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    model.add(Flatten())
    model.add(Dense(4096, activation='sigmoid',
                   kernel_regularizer=l2(1e-3),
                   kernel_initializer=initialize_weights,bias_initializer=initialize_bias))
    '''
    model = Sequential()
    model.add(DenseNet121(weights='imagenet',  
                 include_top=False,
                 pooling='avg'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Activation('sigmoid'))
    '''
    
    '''res = np.load('../res.npz',allow_pickle = True)
    res = dict(res)['wb']
    model.layers[0].set_weights(m.layers[1].get_weights())
    model.layers[0].trainable = False'''
    
    # Generate the encodings (feature vectors) for the two images
    encoded_l = model(left_input)
    encoded_r = model(right_input)
    
    # Add a customized layer to compute the absolute difference between the encodings
    L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])
    L1_distance = BatchNormalization(axis = 1)(L1_distance)
    # Add a dense layer with a sigmoid unit to generate the similarity score
    prediction = Dense(1,activation='sigmoid', bias_initializer=initialize_bias)(L1_distance)
    # Connect the inputs with the outputs
    siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)
    
    # return the model
    return siamese_net

In [8]:
model = get_siamese_model((224,224,3))
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 4096)         420642112   input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 4096)         0           sequential[0][0]      

In [42]:
optimizer = Adam(lr = 1e-4)
#model.layers[2].trainable = False
model.compile(loss="binary_crossentropy",optimizer=optimizer)

In [43]:
def get_batch(batch_size,s="train"):
    """Create batch of n pairs, half same class, half different class"""
    if s == 'train':
        X = Xtrain
        categories = train_classes
    else:
        X = Xval
        categories = val_classes
    n_classes, n_examples, w, h, c = X.shape

    # randomly sample several classes to use in the batch
    categories = rng.choice(n_classes,size=(batch_size,),replace=False)
    
    # initialize 2 empty arrays for the input image batch
    pairs=[np.zeros((batch_size, h, w, 3)) for i in range(2)]
    
    # initialize vector for the targets
    targets=np.zeros((batch_size,))
    
    # make one half of it '1's, so 2nd half of batch has same class
    targets[batch_size//2:] = 1
    for i in range(batch_size):
        category = categories[i]
        idx_1 = rng.randint(0, n_examples)
        pairs[0][i,:,:,:] = X[category, idx_1].reshape(w, h, 3)
        idx_2 = rng.randint(0, n_examples)

        # pick images of same class for 1st half, different for 2nd
        if i >= batch_size // 2:
            category_2 = category  
        else: 
            # add a random number to the category modulo n classes to ensure 2nd image has a different category
            category_2 = (category + rng.randint(1,n_classes)) % n_classes
        
        pairs[1][i,:,:,:] = X[category_2,idx_2].reshape(w, h, 3)
    
    return pairs, targets

In [44]:
def generate(batch_size, s="train"):
    """a generator for batches, so model.fit_generator can be used. """
    while True:
        pairs, targets = get_batch(batch_size,s)
        yield (pairs, targets)

In [81]:
def make_oneshot_task(N, s="val", language=None):
    """Create pairs of test image, support set for testing N way one-shot learning. """
    if s == 'train':
        X = Xtrain
        categories = train_classes
    else:
        X = Xval
        categories = val_classes
    n_classes, n_examples, w, h, c = X.shape
    
    indices = rng.randint(0, n_examples,size=(N,))
    if language is not None: # if language is specified, select characters for that language
        low, high = categories[language]
        if N > high - low:
            raise ValueError("This language ({}) has less than {} letters".format(language, N))
        categories = rng.choice(range(low,high),size=(N,),replace=False)

    else: # if no language specified just pick a bunch of random letters
        categories = rng.choice(range(n_classes),size=(N,),replace=False)            
    true_category = categories[0]
    ex1, ex2 = rng.choice(n_examples,replace=False,size=(2,))
    test_image = np.asarray([X[true_category,ex1,:,:,:]]*N).reshape(N, w, h,3)
    support_set = X[categories,indices,:,:,:]
    support_set[0,:,:,:] = X[true_category,ex2]
    support_set = support_set.reshape(N, w, h,3)
    targets = np.zeros((N,))
    targets[0] = 1
    #targets, test_image, support_set = shuffle(targets, test_image, support_set)
    pairs = [test_image,support_set]
    '''
    out = ux[categories,indices,:,:,:]
    out[0] = ux[true_category,ex2]
    out = out.reshape(N, w, h,3)
    outt = ux[true_category, ex1].reshape(w, h, 3)
    for i in range(N):
        cv2.imwrite(str(i)+'.jpg', out[i])
    cv2.imwrite('t.jpg', outt)
    print(indices, ex1, ex2)
    '''
    return pairs, targets

In [82]:
def test_oneshot(model, N, k, s = "val", verbose = 0):
    """Test average N way oneshot learning accuracy of a siamese neural net over k one-shot tasks"""
    n_correct = 0
    if verbose:
        print("Evaluating model on {} random {} way one-shot learning tasks ... \n".format(k,N))
    for i in range(k):
        inputs, targets = make_oneshot_task(N,s)
        probs = model.predict(inputs)
        print(probs)
        if np.argmax(probs) == np.argmax(targets):
            n_correct+=1
    percent_correct = (100.0 * n_correct / k)
    if verbose:
        print("Got an average of {}% {} way one-shot learning accuracy \n".format(percent_correct,N))
    return percent_correct

In [47]:
# Hyper parameters
evaluate_every = 200 # interval for evaluating on one-shot tasks
batch_size = 23
n_iter = 1000 # No. of training iterations
N_way = 5 # how many classes for testing one-shot tasks
n_val = 30 # how many one-shot tasks to validate on
best = -1

In [83]:
model_path = './weights/'

In [51]:
print("Starting training process!")
print("-------------------------------------")
tf.keras.backend.clear_session()
t_start = time.time()
for i in range(1, n_iter+1):
    (inputs,targets) = get_batch(batch_size)
    loss = model.train_on_batch(inputs, targets)
    if i % evaluate_every == 0:
        print("\n ------------- \n")
        print("Time for {0} iterations: {1} mins".format(i, (time.time()-t_start)/60.0))
        print("Train Loss: {0}".format(loss)) 
        val_acc = test_oneshot(model, N_way, n_val, verbose=True)
        train_acc = test_oneshot(model, N_way, n_val, s='train',verbose=True)
        model.save_weights(os.path.join(model_path, 'weights.{}.h5'.format(i)))
        if val_acc >= best:
            print("Current best: {0}, previous best: {1}".format(val_acc, best))
            best = val_acc

Starting training process!
-------------------------------------

 ------------- 

Time for 200 iterations: 0.5915686845779419 mins
Train Loss: 0.6315654516220093
Evaluating model on 30 random 5 way one-shot learning tasks ... 

Got an average of 50.0% 5 way one-shot learning accuracy 

Evaluating model on 30 random 5 way one-shot learning tasks ... 

Got an average of 93.33333333333333% 5 way one-shot learning accuracy 

Current best: 50.0, previous best: 40.0

 ------------- 

Time for 400 iterations: 1.273361361026764 mins
Train Loss: 0.437203973531723
Evaluating model on 30 random 5 way one-shot learning tasks ... 

Got an average of 50.0% 5 way one-shot learning accuracy 

Evaluating model on 30 random 5 way one-shot learning tasks ... 

Got an average of 96.66666666666667% 5 way one-shot learning accuracy 

Current best: 50.0, previous best: 50.0

 ------------- 

Time for 600 iterations: 1.9575650175412496 mins
Train Loss: 0.47477394342422485
Evaluating model on 30 random 5 way 

In [181]:
test_oneshot(model, 5, 1, s = "val", verbose = 1)

Evaluating model on 1 random 5 way one-shot learning tasks ... 

[ 2  3 12  3  7] 1 0
[[0.9679754 ]
 [0.99317485]
 [0.8225033 ]
 [0.93332195]
 [0.8523745 ]]
Got an average of 0.0% 5 way one-shot learning accuracy 



0.0

In [80]:
model.load_weights(os.path.join(model_path, "weights.1000.h5"))

In [66]:
model.save("2")

INFO:tensorflow:Assets written to: 2\assets
